In [53]:
import pandas as pd
import numpy as np
import geopandas as gpd
import numpy.linalg as la
import os

In [54]:
survey_path = "../../../resources/surveys/egt_2010/survey"
output_path = "../../../results/surveys/egt_2010/cleaned"

In [55]:
if "papermill" in locals():
    survey_path = papermill.input[0]
    output_path = papermill.output[0]

In [56]:
households_path = "{}/Menages_semaine.csv".format(survey_path)
assert os.path.exists(households_path)

persons_path = "{}/Personnes_semaine.csv".format(survey_path)
assert os.path.exists(households_path)

trips_path = "{}/Deplacements_semaine.csv".format(survey_path)
assert os.path.exists(households_path)

legs_path = "{}/Trajets_semaine.csv".format(survey_path)
assert os.path.exists(households_path)

In [57]:
# Read households
df_households = pd.read_csv(
    households_path,
    sep = ",", encoding = "latin1", usecols = [
    "NQUEST", "NB_VD", "NB_VELO", "NB_2RM", "REVENU", "MNP"
])

# Read persons
df_persons = pd.read_csv(
    persons_path,
    sep = ",", encoding = "latin1", usecols = [
        "RESDEP", "NP", "POIDSP", "NQUEST", "SEXE", "AGE", "PERMVP",
        "ABONTC", "OCCP", "PERM2RM", "NBDEPL", "CS8", "PERMVP", "PERM2RM"
    ]
)

# Read trips
df_trips = pd.read_csv(
    trips_path,
    sep = ",", encoding = "latin1", usecols = [
        "NQUEST", "NP", "ND",
        "ORDEP", "DESTDEP", "ORH", "DESTH", "ORM", "DESTM", "ORCOMM", "DESTCOMM",
        "DPORTEE", "MODP_H7", "DESTMOT_H9", "ORMOT_H9",
        "ORCOMM", "DESTCOMM", "DUREE", "ORC", "DESTC",
        "ORMOT", "DESTMOT"
    ]
)

# Read legs
df_legs = pd.read_csv(
    legs_path,
    sep = ",", encoding = "latin1", usecols = [
        "NQUEST", "NP", "ND", "MOYEN", "NT", "TPORTEE"
    ]
)

In [58]:
# Assign IDs to households
df_households = df_households.sort_values(by = "NQUEST")
df_households["household_id"] = np.arange(len(df_households))

# Assign IDs to persons
df_persons = df_persons.sort_values(by = ["NQUEST", "NP"])
df_persons["person_id"] = np.arange(len(df_persons))
df_persons = pd.merge(df_persons, df_households[["NQUEST", "household_id"]], on = "NQUEST")

# Assign IDs to trips
df_trips = df_trips.sort_values(by = ["NQUEST", "NP", "ND"])
df_trips["trip_id"] = np.arange(len(df_trips))
df_trips["trip_sequence"] = df_trips["ND"].astype(int)
df_trips = pd.merge(df_trips, df_persons[["NQUEST", "NP", "person_id", "household_id"]], on = ["NQUEST", "NP"])

# Assign IDs to legs
df_legs = df_legs.sort_values(by = ["NQUEST", "NP", "ND", "NT"])
df_legs["leg_id"] = np.arange(len(df_legs))
df_legs["trip_sequence"] = df_legs["ND"].astype(int)
df_legs["leg_sequence"] = df_legs["NT"].astype(int)
df_legs = pd.merge(df_legs, df_trips[["NQUEST", "NP", "ND", "person_id", "household_id", "trip_id"]], on = ["NQUEST", "NP", "ND"])

### Households

In [59]:
df_households["number_of_cars"] = df_households["NB_VD"]
df_households["number_of_motorbikes"] = df_households["NB_2RM"]
df_households["number_of_bicycles"] = df_households["NB_VELO"]

INCOME_UPPER_BOUNDS = [800, 1200, 1600, 2000, 2400, 3000, 3500, 4500, 5500]
df_households["income_EUR"] = np.nan

for income_class, value in enumerate(INCOME_UPPER_BOUNDS):
    f = df_households["REVENU"] == income_class - 1
    df_households.loc[f, "income_EUR"] = float(INCOME_UPPER_BOUNDS[income_class - 1])

df_households["income_per_person_EUR"] = df_households["income_EUR"] / df_households["MNP"]

df_households = df_households[[
    "household_id", "number_of_cars", 
    "number_of_motorbikes", "number_of_bicycles",
    "income_EUR", "income_per_person_EUR"]]

### Persons

In [60]:
df_persons["weight"] = df_persons["POIDSP"]
df_persons["home_departement_id"] = df_persons["RESDEP"].astype(str)
df_persons["has_driving_permit"] = df_persons["PERMVP"] == 1
df_persons["has_motorbike_permit"] = df_persons["PERM2RM"] == 1
df_persons["has_pt_subscription"] = df_persons["ABONTC"] != 1
df_persons["age"] = df_persons["AGE"].astype(int)

# we assume "small" motorbikes
df_persons["has_motorbike_permit"] |= df_persons["has_driving_permit"]

In [61]:
df_persons = df_persons[[
    "household_id", "person_id", "has_driving_permit", "has_motorbike_permit", "has_pt_subscription", "age", 
    "weight"]]

### Trips

In [62]:
# Trip mode
MODES_MAP = {
    1 : "pt",
    2 : "car",
    3 : "car_passenger",
    4 : "motorbike",
    5 : "bicycle",
    7 : "walk"
}

df_trips["mode"] = "pt"

for category, mode in MODES_MAP.items():
    df_trips.loc[df_trips["MODP_H7"] == category, "mode"] = mode

In [63]:
# Departments
df_trips["origin_departement_id"] = df_trips["ORDEP"].astype(str)
df_trips["destination_departement_id"] = df_trips["DESTDEP"].astype(str)

In [64]:
# Municipalities
df_trips["origin_municipality_id"] = df_trips["ORCOMM"].astype(str)
df_trips["destination_municipality_id"] = df_trips["DESTCOMM"].astype(str)

In [65]:
# Further trip attributes
df_trips["euclidean_distance"] = df_trips["DPORTEE"] * 1000.0

In [66]:
df_trips["departure_time"] = df_trips["ORH"] * 3600 + df_trips["ORM"] * 60
df_trips["travel_time"] = df_trips["DUREE"] * 60

In [67]:
df_trips["origin_cell"] = df_trips["ORC"]
df_trips["destination_cell"] = df_trips["DESTC"]

In [68]:
ACTIVITY_TYPE_MAP = {
    "home": (1,),
    "work": (2,3),
    "education": (4,),
    "shop": (5,),
    "leisure": (8,)
}

In [69]:
df_trips["origin_activity_type"] = "other"
df_trips["destination_activity_type"] = "other"

for activity_type, values in ACTIVITY_TYPE_MAP.items():
    df_trips.loc[df_trips["ORMOT_H9"].isin(values), "origin_activity_type"] = activity_type
    df_trips.loc[df_trips["DESTMOT_H9"].isin(values), "destination_activity_type"] = activity_type

In [70]:
df_trips = df_trips[[
    "household_id", "person_id", "trip_id", "trip_sequence", "mode", "euclidean_distance",
    "origin_municipality_id", "destination_municipality_id", "travel_time", "departure_time",
    "origin_cell", "destination_cell", "origin_activity_type", "destination_activity_type"
]]

### Legs

In [71]:
# Leg mode
LEG_MODES_MAP = {
    1 : ("walk", "walk"),
    
    10 : ("pt", "rail"),
    11 : ("pt", "rail"),
    12 : ("pt", "funicular"),
    13 : ("pt", "subway"),
    14 : ("pt", "tram"),
    15 : ("pt", "bus"),
    16 : ("pt", "bus"),
    17 : ("pt", "bus"),
    18 : ("pt", "bus"),
    19 : ("pt", "bus"),

    20 : ("pt", "other"),
    30 : ("pt", "other"),
    31 : ("pt", "other"),
    32 : ("pt", "other"),
    33 : ("pt", "other"),
    34 : ("pt", "other"),
    
    35 : ("pt", "other"),

    40 : ("pt", "other"),
    41 : ("pt", "rail"),
    42 : ("pt", "rail"),
    43 : ("pt", "rail"),

    50 : ("car", "car"),
    51 : ("car", "car"),
    52 : ("car", "car"),
    53 : ("car", "car"),

    54 : ("car", "car"),
    55 : ("car", "car"),

    60 : ("bike", "bike"),
    61 : ("bike", "bike"),
    62 : ("bike", "bike"),
    63 : ("bike", "bike"),

    70 : ("car_passenger", "car_passenger"),
    71 : ("car_passenger", "car_passenger"),
    72 : ("car_passenger", "car_passenger"),
    73 : ("car_passenger", "car_passenger"),

    74 : ("car_passenger", "car_passenger"),
    75 : ("car_passenger", "car_passenger"),

    80 : ("other", "other"),
    81 : ("other", "other"),
    82 : ("other", "other"),
}

df_legs["mode"] = "other"
df_legs["transit_mode"] = "other"
df_legs["mode_code"] = df_legs["MOYEN"]

for category, (leg_mode, transit_mode) in LEG_MODES_MAP.items():
    df_legs.loc[df_legs["MOYEN"] == category, "mode"] = leg_mode
    df_legs.loc[df_legs["MOYEN"] == category, "transit_mode"] = transit_mode

In [72]:
df_legs = df_legs[["household_id", "person_id", "trip_id", "leg_id", "trip_sequence", "leg_sequence",
    "mode", "transit_mode", "mode_code"]]

### Checking for NaN

In [73]:
# Households, persons, legs don't have any NaNs
# Trips has some NaNs for trips that go out of the region, we add a dummy to indicate that

df_trips["is_valid"] = True

for column in df_trips.columns:
    f = df_trips[column].isna()
    df_trips.loc[f, "is_valid"] = False

    if np.count_nonzero(f) > 0:
        print(column, np.count_nonzero(f))

euclidean_distance 926
travel_time 1027
departure_time 71
origin_cell 536
destination_cell 353


### Output

In [74]:
df_households.to_parquet("{}/households.parquet".format(output_path))
df_persons.to_parquet("{}/persons.parquet".format(output_path))
df_trips.to_parquet("{}/trips.parquet".format(output_path))
df_legs.to_parquet("{}/legs.parquet".format(output_path))